# 使用人工神经网络求解微分方程问题

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.optimize import minimize
import scipy as sp
# 目标函数
# 定义目标函数
def targetfunction(x,xs,n,m):# x是nparray,xs是nparray，n和m是number
    result=0
    omega=x[0:n]
    miu=x[n:2*n]
    theta=x[2*n:3*n]
    for i in range(m):
        for j in range(n):
            result+=(2*miu[j]/(1+math.exp(theta[j]-omega[j]*xs[i]))+xs[i]*miu[j]*omega[j]*math.exp(theta[j]-omega[j]*xs[i])/(1+math.exp(theta[j]-omega[j]*xs[i]))**2-miu[j]/xs[i]/(1+math.exp(theta[j]-omega[j])))
        result-=xs[i]**3
        result+=(2/(5*xs[i]))
        result=result**2
    return result
# 求梯度
def asgrad(x,xs,epsilon,n,m):#近似的梯度
    result=[]
    xtemp=x.copy()
    for i in range(3*n):
        xtemp[i]=xtemp[i]+epsilon
        calc=(targetfunction(xtemp,xs,n,m)-targetfunction(x,xs,n,m))/epsilon
        result.append(calc)
        xtemp=x.copy()
    return result
# 精确解
def precisesolve(x):# 精确解
    return x**4/5+1/(5*x)

def equation(x,y):
    return x**3-y/x  # 第一个方程dy/dx

def grad(func,xk):
    '''
    :param func: 目标函数
    :param xk: 列向量
    :return: func在xk处梯度grad(数值估计方法）
    '''
    delta=0.00001
    grad=[]
    n=np.size(xk)
    for i in range(0,n):
        x=[]
        for j in range(0,n):
            x.append(xk[j,0])
        x[i]=xk[i,0]+delta # 第i个分量微变
        grad.append((func(np.mat(x).T)-func(xk))/delta)
    return np.mat(grad).T


def phi(x,omega,theta):
    # Sigmoid型激活函数phiz
    z = omega*x-theta
    return 1/(1+math.exp(-z))

def yt(x,p):
    # 前馈神经网络输出yt
    n = int(np.size(p)/3)
    N = 0 # 初始条件
    for i in range(0,n):
        N += p[n+i,0]*phi(x,p[i,0],p[2*n+i,0])
    return N

def opti_prob(func,p,m):
    # 导出最优化问题res_prob
    delta = 0.0001
    x = []
    gradx = []  # 对x的偏导
    res_prob = 0
    # 取区间【1,2】上均匀的值
    for i in range(0,m):
        x.append(1+i/(m+1))
        gradx.append((yt(x[i]+delta,p)-yt(x[i],p))/delta)
        res_prob += (gradx[i]-func(x[i],yt(x[i],p)))**2
    return np.float64(res_prob)

# 拟牛顿法
def asnewton(x,xs,method,epsilon,n,m):#x是一个nparray
    # 变量
    H=np.eye(3*n)
    grad=asgrad(x,xs,1e-6,3,3)
    time=0
    gk=0
    yk=0
    sk=0
    #yk=gk+1-gk,sk=sk+1-sk
    while True:
        gk=grad
        
        #终止条件
        if np.linalg.norm(grad)<=epsilon:
            break
            
        #计算方向
        dk=-np.mat(H)*np.mat(grad).transpose()
        
        # 线搜索armijo方法，用于对方向上进行线搜索，没改
        alpha = 1
        while targetfunction((x+alpha*dk.transpose()).A[0].tolist(),xs,n,m)>targetfunction(x,xs,n,m)+0.5*alpha*np.dot(grad,dk):
            alpha = alpha/2
        x = x+alpha*dk.transpose()
        
        #更新sk,yk
        grad=asgrad(x,xs,1e-6,3,3)
        sk= alpha*dk
        yk= grad-gk
        
        #更新H
        if method=='DFP':
            H = H+(sk.dot(sk.T))/np.linalg.norm(sk.T.dot(yk))-(H.dot(yk).dot(yk.T).dot(H))/np.linalg.norm(yk.T.dot(H).dot(yk))
        elif method=='BFGS':
            H = H+(1+np.linalg.norm((yk.T.dot(H).dot(yk))/(yk.T.dot(sk))))*((sk.dot(sk.T))/np.linalg.norm(yk.T.dot(sk)))-((sk.dot(yk.T).dot(H))+(H.dot(yk).dot(sk.T)))/np.linalg.norm(yk.T.dot(sk))
        else:
            print('error')
            
        #更新迭代次数
        time+=1
    print(f"{method}迭代次数：{time}")
    return x

x=asnewton([1,1,1,1,1,1,1,1,1],[1,1.5,2],'BFGS',1e-6,3,3)
print(x)
# 实验解
def experimentalsolve(x,omega,miu,theta,n):
    result=0
    for j in range(n):
        result+=x*miu[j]/(1+math.exp(theta[j]-omega[j]*x))
        result-=miu[j]/(1+math.exp(theta[j]-omega[j]))
    result+=0.4
    return result
# 变量定义
ma=4
na=4
p=[]

omegaD=[]
thetaD=[]
miuD=[]

omegaB=[]
thetaB=[]
miuB=[]

X=np.linspace(1,2,ma).reshape(1,ma)

for i in range(3*na):
   p.append(1) 
print(X)
print(p)
# 拟牛顿方法输出omega，theta，miu的参数
p=asnewton(p,X,'DFP',1e-6)
for i in range(na):
    omegaD.append(p[i])
    thetaD.append(p[i+na])
    miuD.append(p[i+na+na])
p=asnewton(p,X,p,'BFGS',1e-6)
for i in range(na):
    omegaB.append(p[i])
    thetaB.append(p[i+na])
    miuB.append(p[i+na+na])

# 可视化
# 绘图
Xgraph=np.linspace(1,2,100)
Ygraph=[precisesolve(i) for i in Xgraph]
Ypredict1=[experimentalsolve(i,omegaD,thetaD,miuD) for i in Xgraph]
Ypredict2=[experimentalsolve((i,omegaB,thetaB,miuB) for i in Xgraph)]
plt.plot(Xgraph,Ygraph)
plt.plot(Xgraph,Ypredict1)
plt.plot(Xgraph,Ypredict2)
plt.legend("precise solve","DFP","BFGS")
plt.title("DFP and BFGS solving square")
plt.show()

OverflowError: (34, 'Result too large')